In [11]:
import pandas as pd
import numpy as np

# ============================================================
# LOAD CLEANED DATASETS
# ============================================================

print("\n" + "="*70)
print(" LOADING CLEANED DATASETS")
print("="*70)

# Load cleaned data
z1_clean = pd.read_csv('data/cleaned/z1_cajica_cleaned.csv', index_col='Date', parse_dates=True)
z2_clean = pd.read_csv('data/cleaned/z2_giralda_cleaned.csv', index_col='Date', parse_dates=True)
z3_clean = pd.read_csv('data/cleaned/z3_oikos_cleaned.csv', index_col='Date', parse_dates=True)

print("\n✅ Cleaned datasets loaded:")
print(f"   Z1_CAJICA:  {len(z1_clean):>6} records, {len(z1_clean.columns):>2} columns")
print(f"   Z2_GIRALDA: {len(z2_clean):>6} records, {len(z2_clean.columns):>2} columns")
print(f"   Z3_OIKOS:   {len(z3_clean):>6} records, {len(z3_clean.columns):>2} columns")

print("="*70 + "\n")



 LOADING CLEANED DATASETS

✅ Cleaned datasets loaded:
   Z1_CAJICA:   43961 records, 17 columns
   Z2_GIRALDA:  44030 records, 13 columns
   Z3_OIKOS:    42347 records, 17 columns



In [12]:

# ============================================================
# STEP 1: SYNCHRONIZE STATIONS TO COMMON TIMESTAMPS
# ============================================================

def synchronize_stations(zona1_clean, zona2_clean, zona3_clean):
    """
    Synchronize the 3 stations to common timestamps
    Required for barycentric interpolation
    """
    print("\n" + "="*70)
    print(" STEP 1: SYNCHRONIZING STATIONS")
    print("="*70)
    
    # Find common timestamps (intersection)
    common_timestamps = zona1_clean.index.intersection(
        zona2_clean.index
    ).intersection(zona3_clean.index)
    
    print(f"\nOriginal records:")
    print(f"  Z1_CAJICA:  {len(zona1_clean):>6} records")
    print(f"  Z2_GIRALDA: {len(zona2_clean):>6} records")
    print(f"  Z3_OIKOS:   {len(zona3_clean):>6} records")
    print(f"\nCommon timestamps: {len(common_timestamps):>6} records")
    
    # Calculate data loss
    max_records = max(len(zona1_clean), len(zona2_clean), len(zona3_clean))
    data_loss = (1 - len(common_timestamps)/max_records) * 100
    print(f"Data loss: {data_loss:.2f}%")
    
    # Align dataframes
    z1_sync = zona1_clean.loc[common_timestamps].copy()
    z2_sync = zona2_clean.loc[common_timestamps].copy()
    z3_sync = zona3_clean.loc[common_timestamps].copy()
    
    print(f"\n✅ Stations synchronized to {len(common_timestamps)} records")
    print("="*70 + "\n")
    
    return z1_sync, z2_sync, z3_sync

z1_sync, z2_sync, z3_sync = synchronize_stations(z1_clean, z2_clean, z3_clean)




 STEP 1: SYNCHRONIZING STATIONS

Original records:
  Z1_CAJICA:   43961 records
  Z2_GIRALDA:  44030 records
  Z3_OIKOS:    42347 records

Common timestamps:  40112 records
Data loss: 8.90%

✅ Stations synchronized to 40112 records



In [13]:

# ============================================================
# STEP 2: HANDLE CRITICAL MISSING VALUES
# ============================================================

def handle_critical_missing_values(z1, z2, z3):
    """
    Remove timestamps where any station has missing critical wind data
    Barycentric interpolation requires complete data from all 3 stations
    """
    print("\n" + "="*70)
    print(" STEP 2: HANDLING CRITICAL MISSING VALUES")
    print("="*70)
    
    critical_vars = ['Wind Speed (m/sec)', 'Wind Direction (°)']
    
    # Check if any station has missing values at synchronized timestamps
    z1_missing = z1[critical_vars].isnull().any(axis=1)
    z2_missing = z2[critical_vars].isnull().any(axis=1)
    z3_missing = z3[critical_vars].isnull().any(axis=1)
    
    any_missing = z1_missing | z2_missing | z3_missing
    
    print(f"\nTimestamps with missing critical data:")
    print(f"  Z1_CAJICA:  {z1_missing.sum():>6} timestamps")
    print(f"  Z2_GIRALDA: {z2_missing.sum():>6} timestamps")
    print(f"  Z3_OIKOS:   {z3_missing.sum():>6} timestamps")
    print(f"  Total:      {any_missing.sum():>6} timestamps")
    
    if any_missing.sum() > 0:
        removal_pct = (any_missing.sum() / len(z1)) * 100
        print(f"\n⚠️  Removing {any_missing.sum()} timestamps ({removal_pct:.2f}%)")
        print(f"   Reason: Barycentric interpolation requires data from all 3 stations")
        
        # Remove timestamps with any missing critical data
        z1_final = z1[~any_missing].copy()
        z2_final = z2[~any_missing].copy()
        z3_final = z3[~any_missing].copy()
        
        print(f"\n✅ Final synchronized dataset: {len(z1_final)} records")
    else:
        print(f"\n✅ No missing critical values - all {len(z1)} records are usable")
        z1_final, z2_final, z3_final = z1, z2, z3
    
    print("="*70 + "\n")
    
    return z1_final, z2_final, z3_final

z1_final, z2_final, z3_final = handle_critical_missing_values(z1_sync, z2_sync, z3_sync)




 STEP 2: HANDLING CRITICAL MISSING VALUES

Timestamps with missing critical data:
  Z1_CAJICA:     313 timestamps
  Z2_GIRALDA:      0 timestamps
  Z3_OIKOS:        7 timestamps
  Total:         320 timestamps

⚠️  Removing 320 timestamps (0.80%)
   Reason: Barycentric interpolation requires data from all 3 stations

✅ Final synchronized dataset: 39792 records



In [14]:

# ============================================================
# STEP 3: CREATE COMBINED DATASET
# ============================================================

def create_combined_dataset(z1, z2, z3):
    """
    Combine the 3 stations into a single wide-format dataset
    Each station's variables get a suffix (_z1, _z2, _z3)
    """
    print("\n" + "="*70)
    print(" STEP 3: CREATING COMBINED DATASET")
    print("="*70)
    
    # Reset index to have Date as column
    z1_reset = z1.reset_index()
    z2_reset = z2.reset_index()
    z3_reset = z3.reset_index()
    
    # Keep only one Date column
    date_col = z1_reset[['Date']].copy()
    
    # Remove Date from individual stations
    z1_data = z1_reset.drop('Date', axis=1)
    z2_data = z2_reset.drop('Date', axis=1)
    z3_data = z3_reset.drop('Date', axis=1)
    
    # Add suffixes to distinguish stations
    z1_data.columns = [f"{col}_z1" for col in z1_data.columns]
    z2_data.columns = [f"{col}_z2" for col in z2_data.columns]
    z3_data.columns = [f"{col}_z3" for col in z3_data.columns]
    
    # Combine all dataframes
    zones_combined = pd.concat([date_col, z1_data, z2_data, z3_data], axis=1)
    
    print(f"\n✅ Combined dataset created")
    print(f"   Records: {len(zones_combined)}")
    print(f"   Columns: {len(zones_combined.columns)}")
    print(f"   Stations: 3 (Z1_CAJICA, Z2_GIRALDA, Z3_OIKOS)")
    
    # Display column structure
    print(f"\nColumn structure:")
    z1_cols = [col for col in zones_combined.columns if col.endswith('_z1')]
    z2_cols = [col for col in zones_combined.columns if col.endswith('_z2')]
    z3_cols = [col for col in zones_combined.columns if col.endswith('_z3')]
    
    print(f"  Date column: 1")
    print(f"  Z1_CAJICA:   {len(z1_cols)} variables")
    print(f"  Z2_GIRALDA:  {len(z2_cols)} variables")
    print(f"  Z3_OIKOS:    {len(z3_cols)} variables")
    
    print("="*70 + "\n")
    
    return zones_combined

zones_combined = create_combined_dataset(z1_final, z2_final, z3_final)



 STEP 3: CREATING COMBINED DATASET

✅ Combined dataset created
   Records: 39792
   Columns: 48
   Stations: 3 (Z1_CAJICA, Z2_GIRALDA, Z3_OIKOS)

Column structure:
  Date column: 1
  Z1_CAJICA:   17 variables
  Z2_GIRALDA:  13 variables
  Z3_OIKOS:    17 variables



In [15]:


# ============================================================
# STEP 4: ADD TEMPORAL FEATURES
# ============================================================

def add_temporal_features(df):
    """
    Add time-based features useful for analysis and forecasting
    """
    print("\n" + "="*70)
    print(" STEP 4: ADDING TEMPORAL FEATURES")
    print("="*70)
    
    df = df.copy()
    
    # Ensure Date is datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Extract temporal components
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['day'] = df['Date'].dt.day
    df['hour'] = df['Date'].dt.hour
    df['minute'] = df['Date'].dt.minute
    df['day_of_week'] = df['Date'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['day_of_year'] = df['Date'].dt.dayofyear
    df['week_of_year'] = df['Date'].dt.isocalendar().week
    
    # Create categorical features
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['is_daytime'] = ((df['hour'] >= 6) & (df['hour'] <= 18)).astype(int)
    
    # Create time of day categories
    def categorize_time_of_day(hour):
        if 6 <= hour < 12:
            return 'morning'
        elif 12 <= hour < 18:
            return 'afternoon'
        elif 18 <= hour < 22:
            return 'evening'
        else:
            return 'night'
    
    df['time_of_day'] = df['hour'].apply(categorize_time_of_day)
    
    print(f"\n✅ Temporal features added:")
    print(f"   - year, month, day, hour, minute")
    print(f"   - day_of_week, day_of_year, week_of_year")
    print(f"   - is_weekend, is_daytime")
    print(f"   - time_of_day (morning/afternoon/evening/night)")
    
    print("="*70 + "\n")
    
    return df

zones_combined = add_temporal_features(zones_combined)



 STEP 4: ADDING TEMPORAL FEATURES

✅ Temporal features added:
   - year, month, day, hour, minute
   - day_of_week, day_of_year, week_of_year
   - is_weekend, is_daytime
   - time_of_day (morning/afternoon/evening/night)



In [16]:


# ============================================================
# STEP 5: ADD WIND VECTOR COMPONENTS (CRITICAL FOR INTERPOLATION)
# ============================================================

def add_wind_vectors(df):
    """
    Convert wind speed and direction to vector components (U, V)
    This is ESSENTIAL for barycentric interpolation
    """
    print("\n" + "="*70)
    print(" STEP 5: ADDING WIND VECTOR COMPONENTS")
    print("="*70)
    
    df = df.copy()
    
    stations = ['z1', 'z2', 'z3']
    
    for station in stations:
        speed_col = f'Wind Speed (m/sec)_{station}'
        dir_col = f'Wind Direction (°)_{station}'
        
        if speed_col in df.columns and dir_col in df.columns:
            # Convert direction to radians
            dir_rad = np.radians(df[dir_col])
            
            # Calculate vector components
            # U component (east-west): positive towards east
            df[f'wind_u_{station}'] = df[speed_col] * np.sin(dir_rad)
            
            # V component (north-south): positive towards north
            df[f'wind_v_{station}'] = df[speed_col] * np.cos(dir_rad)
            
            # Also encode direction as circular (for analysis/visualization)
            df[f'wind_dir_sin_{station}'] = np.sin(dir_rad)
            df[f'wind_dir_cos_{station}'] = np.cos(dir_rad)
            
            print(f"   ✓ {station.upper()}: wind_u, wind_v, wind_dir_sin, wind_dir_cos")
    
    print(f"\n✅ Wind vector components added for all 3 stations")
    print(f"   Use U,V components for spatial interpolation")
    print(f"   Use sin,cos for circular statistics on direction")
    
    print("="*70 + "\n")
    
    return df

zones_combined = add_wind_vectors(zones_combined)



 STEP 5: ADDING WIND VECTOR COMPONENTS
   ✓ Z1: wind_u, wind_v, wind_dir_sin, wind_dir_cos
   ✓ Z2: wind_u, wind_v, wind_dir_sin, wind_dir_cos
   ✓ Z3: wind_u, wind_v, wind_dir_sin, wind_dir_cos

✅ Wind vector components added for all 3 stations
   Use U,V components for spatial interpolation
   Use sin,cos for circular statistics on direction



In [17]:


# ============================================================
# STEP 6: ADD DERIVED VARIABLES
# ============================================================

def add_derived_variables(df):
    """
    Add useful derived variables for analysis
    """
    print("\n" + "="*70)
    print(" STEP 6: ADDING DERIVED VARIABLES")
    print("="*70)
    
    df = df.copy()
    
    stations = ['z1', 'z2', 'z3']
    
    for station in stations:
        speed_col = f'Wind Speed (m/sec)_{station}'
        gust_col = f'Wind Gust (m/sec)_{station}'
        
        if speed_col in df.columns and gust_col in df.columns:
            # Gust factor (turbulence indicator)
            df[f'gust_factor_{station}'] = df[gust_col] / (df[speed_col] + 0.01)
            
            # Wind speed change rate (for detecting fronts)
            df[f'wind_speed_change_{station}'] = df[speed_col].diff()
            df[f'wind_speed_change_rate_{station}'] = df[f'wind_speed_change_{station}'] / 5  # per minute
            
            print(f"   ✓ {station.upper()}: gust_factor, wind_speed_change, wind_speed_change_rate")
    
    print(f"\n✅ Derived variables added for all 3 stations")
    
    print("="*70 + "\n")
    
    return df

zones_combined = add_derived_variables(zones_combined)




 STEP 6: ADDING DERIVED VARIABLES
   ✓ Z1: gust_factor, wind_speed_change, wind_speed_change_rate
   ✓ Z2: gust_factor, wind_speed_change, wind_speed_change_rate
   ✓ Z3: gust_factor, wind_speed_change, wind_speed_change_rate

✅ Derived variables added for all 3 stations



In [18]:

# ============================================================
# STEP 7: FINAL SUMMARY
# ============================================================

def display_final_summary(df):
    """
    Display comprehensive summary of the prepared dataset
    """
    print("\n" + "="*70)
    print(" FINAL DATASET SUMMARY")
    print("="*70)
    
    print(f"\n📊 Dataset Dimensions:")
    print(f"   Records:  {len(df):>8,}")
    print(f"   Columns:  {len(df.columns):>8}")
    
    print(f"\n📅 Time Coverage:")
    print(f"   Start:    {df['Date'].min()}")
    print(f"   End:      {df['Date'].max()}")
    print(f"   Duration: {(df['Date'].max() - df['Date'].min()).days} days")
    
    print(f"\n🌡️ Variable Categories:")
    
    # Count by category
    original_vars = len([col for col in df.columns if any(col.endswith(f'_{s}') for s in ['z1','z2','z3']) 
                        and not any(x in col for x in ['wind_u', 'wind_v', 'wind_dir_sin', 'wind_dir_cos', 
                                                        'gust_factor', 'wind_speed_change', '_is_missing'])])
    vector_vars = len([col for col in df.columns if 'wind_u' in col or 'wind_v' in col 
                       or 'wind_dir_sin' in col or 'wind_dir_cos' in col])
    derived_vars = len([col for col in df.columns if 'gust_factor' in col or 'wind_speed_change' in col])
    temporal_vars = len([col for col in df.columns if col in ['year', 'month', 'day', 'hour', 'minute', 
                                                                'day_of_week', 'day_of_year', 'week_of_year',
                                                                'is_weekend', 'is_daytime', 'time_of_day']])
    flag_vars = len([col for col in df.columns if '_is_missing' in col or 'warning' in col or 'critical' in col])
    
    print(f"   Original meteorological:  {original_vars:>3} variables")
    print(f"   Wind vector components:   {vector_vars:>3} variables")
    print(f"   Derived variables:        {derived_vars:>3} variables")
    print(f"   Temporal features:        {temporal_vars:>3} variables")
    print(f"   Quality flags:            {flag_vars:>3} variables")
    print(f"   Date column:              {1:>3} column")
    print(f"   {'─'*35}")
    print(f"   TOTAL:                    {len(df.columns):>3} columns")
    
    print(f"\n✅ Missing Values Check:")
    total_missing = df.isnull().sum().sum()
    if total_missing == 0:
        print(f"   No missing values! ✓")
    else:
        print(f"   Total missing values: {total_missing}")
        missing_cols = df.isnull().sum()
        print(missing_cols[missing_cols > 0])
    
    print(f"\n💾 Ready for EDA and Spatial Interpolation!")
    print("="*70 + "\n")

display_final_summary(zones_combined)




 FINAL DATASET SUMMARY

📊 Dataset Dimensions:
   Records:    39,792
   Columns:        80

📅 Time Coverage:
   Start:    2025-06-04 00:00:00-05:00
   End:      2025-11-04 23:55:00-05:00
   Duration: 153 days

🌡️ Variable Categories:
   Original meteorological:   39 variables
   Wind vector components:    12 variables
   Derived variables:          9 variables
   Temporal features:         11 variables
   Quality flags:             20 variables
   Date column:                1 column
   ───────────────────────────────────
   TOTAL:                     80 columns

✅ Missing Values Check:
   Total missing values: 6
wind_speed_change_z1         1
wind_speed_change_rate_z1    1
wind_speed_change_z2         1
wind_speed_change_rate_z2    1
wind_speed_change_z3         1
wind_speed_change_rate_z3    1
dtype: int64

💾 Ready for EDA and Spatial Interpolation!



In [19]:

# ============================================================
# STEP 8: SAVE PREPARED DATASET
# ============================================================

print("=" * 70)
print(" SAVING PREPARED DATASET")
print("=" * 70)

# Save the final prepared dataset
zones_combined.to_csv('data/prepared/zones_combined_prepared.csv', index=False)

print("\n✅ Prepared dataset saved:")
print("   - data/prepared/zones_combined_prepared.csv")
print(f"   - {len(zones_combined):,} records")
print(f"   - {len(zones_combined.columns)} columns")

# Also assign to zones_combined_cleaned for consistency with your naming
zones_combined_cleaned = zones_combined.copy()

print("\n✅ Dataset ready for EDA in variable: zones_combined_cleaned")
print("=" * 70 + "\n")

# Display first few rows
print("First 5 rows preview:")
print(zones_combined_cleaned.head())

 SAVING PREPARED DATASET

✅ Prepared dataset saved:
   - data/prepared/zones_combined_prepared.csv
   - 39,792 records
   - 80 columns

✅ Dataset ready for EDA in variable: zones_combined_cleaned

First 5 rows preview:
                       Date       Simple Date_z1  Outdoor Temperature (°C)_z1  \
0 2025-06-04 00:00:00-05:00  2025-06-04 00:00:00                         13.2   
1 2025-06-04 00:05:00-05:00  2025-06-04 00:05:00                         13.2   
2 2025-06-04 00:10:00-05:00  2025-06-04 00:10:00                         13.0   
3 2025-06-04 00:15:00-05:00  2025-06-04 00:15:00                         12.9   
4 2025-06-04 00:20:00-05:00  2025-06-04 00:20:00                         12.8   

   Wind Speed (m/sec)_z1  Wind Gust (m/sec)_z1  Max Daily Gust (m/sec)_z1  \
0                    0.5                   1.0                        9.7   
1                    0.1                   0.5                        1.0   
2                    0.4                   0.5                 